# Projeto 1 - Ciência dos Dados

Nome: Pedro Henrique Almeida Vidal  

Nome: Felipe Assed Carvalho

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\Pedro Vidal\Desktop\P1_Cdados_Pedro_Felipe\23-1a-cd-p1-pedro_felipe\notebooks


Carregando a base de dados com os tweets classificados manualmente:

In [3]:
filename = '../data/dados.xls'

In [4]:
train = pd.read_excel(filename)
train.head(5)

,app_name,review,Target
0,Warhammer: Vermintide 2,"jogo não é ruim, mas se eu pudesse eu voltava ...",2.0
1,No Man's Sky,"Não posso avaliar, porque NÃO CONSIGO JOGAR!\n...",2.0
2,The Binding of Isaac,"UPDATE: Dei outra chance, comprei o Rebirth e ...",2.0
3,Among Us,"Parabens aos desenvolvedores desse jogo, muito...",2.0
4,The Elder Scrolls Online,É feio\nNão tem akele girocóptero em camera le...,2.0


In [5]:
train = pd.read_excel(filename, sheet_name = 'Treinamento')
train.head(5)

,app_name,review,Target
0,Warhammer: Vermintide 2,"jogo não é ruim, mas se eu pudesse eu voltava ...",2.0
1,No Man's Sky,"Não posso avaliar, porque NÃO CONSIGO JOGAR!\n...",2.0
2,The Binding of Isaac,"UPDATE: Dei outra chance, comprei o Rebirth e ...",2.0
3,Among Us,"Parabens aos desenvolvedores desse jogo, muito...",2.0
4,The Elder Scrolls Online,É feio\nNão tem akele girocóptero em camera le...,2.0


In [6]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,app_name,review,Target
0,Mount & Blade: Warband,"Mano, mt bom, bora pro mount and blade 2 banne...",1
1,The Witcher 3: Wild Hunt,"Melhor jogo ever do mundo, compra logo isso!",1
2,Rocket League,"Esse Purgatório em forma de jogo, não me troux...",2
3,Grand Theft Auto V,"Não precisa nem comentar né, afinal de contas ...",1
4,PLAYERUNKNOWN'S BATTLEGROUNDS,"O jogo é muito bom, muito bom mesmo, mas infel...",2


___
## Classificador automático


Faça aqui uma descrição do seu assunto e o que considerou como relevante ou não relevante na classificação dos tweets (Target).

Tendo como assunto reviews de jogos na plataforma "Steam", fizemos a seguinte triagem:
1 - Opiniões sem teor julgador negativo
2 - Opiniões anti-recomendativas

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [7]:
#De início, faz-se a limpeza. Para isso, define-se a função cleanup (apresentada na aula 08 - Lusíadas e Dom Casmurro) para que ocorram mudanças de sinais para strings vazias. 
import re
def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[´"!-.:?;$'']' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed
#Em seguida, pensando nos caracteres especiais, realiza-se a seguinte sequência de códigos para que sejam removidos:
def acentos_especiais (texto):
    lista_acentos = ['ã', 'á', 'à', 'â', 'ô', 'õ', 'ó', 'ò', 'ê','è', 'é', 'í', 'ì', 'ú', 'ù']
    vogais = ['a', 'o', 'e', 'i', 'u']
    for caractere in lista_acentos:
        if caractere in texto:
            indexx = texto.index(caractere)
            if indexx <= 3:
                texto = texto.replace(caractere, vogais[0])
            elif indexx > 3 and indexx <= 7:
                texto = texto.replace(caractere, vogais[1])
            elif indexx > 7 and indexx <= 10:
                texto = texto.replace(caractere, vogais[2])
            elif indexx > 10 and indexx <= 12:
                texto = texto.replace(caractere, vogais[3])
            else:
                texto = texto.replace(caractere, vogais[4])
    texto = texto.replace(u'\xa0', '')
    texto = texto.replace(u'\u200b', '')
    texto = texto.replace(u'\\', '')
    texto = texto.replace(u'\u200b', '')
    texto = texto.replace('ç', 'c')
    texto = texto.replace(u'\u2019', '')
    texto = texto.replace(u'\u201D', '')
    texto = texto.replace(u'\u201C', '')
    return texto

In [8]:
#Agora aplica-se ambas funções acima e a função .lower visando a facilitar o manuseio das probabilidades
dados_treinamento = train
dados_treinamento.review = dados_treinamento.review.apply(str).apply(cleanup)
dados_treinamento.review = dados_treinamento.review.apply(acentos_especiais)
dados_treinamento.review = dados_treinamento.review.apply(str.lower)
dados_treinamento

,app_name,review,Target
0,Warhammer: Vermintide 2,jogo noo e ruim mas se eu pudesse eu voltava n...,2.0
1,No Man's Sky,nao posso avaliar porque não consigo jogar\njo...,2.0
2,The Binding of Isaac,update dei outra chance comprei o rebirth e su...,2.0
3,Among Us,parabens aos desenvolvedores desse jogo muito ...,2.0
4,The Elder Scrolls Online,é feio\nneo tem akele girocuptero em camera le...,2.0
...,...,...,...
474,No Man's Sky,░░░░░░░░░░░█▀▀░░█░░░░░░\n░░░░░░▄▀▀▀▀░░░░░█▄▄░░...,2.0
475,PLAYERUNKNOWN'S BATTLEGROUNDS,jogo roda meio pesado ainda e as atualizacues ...,2.0
476,PLAYERUNKNOWN'S BATTLEGROUNDS,a proposta do jogo u utima a execucuo decepcio...,2.0
477,No Man's Sky,it wont open patch it up hello im trying here ...,2.0


In [9]:
#Agora para a planilha de teste:
dados_teste = test
dados_teste.review = dados_teste.review.apply(str).apply(cleanup)
dados_teste.review = dados_teste.review.apply(acentos_especiais)
dados_teste.review = dados_teste.review.apply(str.lower)
dados_teste


,app_name,review,Target
0,Mount & Blade: Warband,mano mt bom bora pro mount and blade 2 bannerlord,1
1,The Witcher 3: Wild Hunt,melhor jogo ever do mundo compra logo isso,1
2,Rocket League,esse purgatirio em forma de jogo nuo me trouxe...,2
3,Grand Theft Auto V,nao precisa nem comentar nu afinal de contas g...,1
4,PLAYERUNKNOWN'S BATTLEGROUNDS,o jogo o muito bom muito bom mesmo mas infeliz...,2
...,...,...,...
316,DARK SOULS™ III,porqueira do caralho nuo colocaram nem a porra...,2
317,Hollow Knight,hollow knight \n\nopiniuo atualmente meu jogo ...,2
318,The Forest,jogo muito ficil e com poucas coisas para fazer,1
319,Grand Theft Auto V,este jogo e maravilhoso vale cada centavo pore...,1


In [10]:
#Pensando na análise e classificação das reviews, temos:
dados_treinamento['Target'] = dados_treinamento['Target'].astype('category')
#Classificações das reviews
dados_treinamento['Target'].cat.categories = ['Ok', 'Negativo']
dados_treinamento['Target'].cat.categories

Index(['Ok', 'Negativo'], dtype='object')

In [11]:
#Agora, visando aos cálculos de probabilidade (e análise de frequência das palavras), temos:
#Divide-se agora as opiniões em 'ok' -> pensando nas avaliações positivas e neureas e 'negativas'
ok = ''
opiniao_ok = dados_treinamento.loc[dados_treinamento.Target == 'Ok',['review']]
for opiniao in opiniao_ok.review:
    ok += opiniao + ''
ok


'targeted\nataca com numeros\nnuo ataca com o mouse\nmobilidade totalmente exquisita joguei 12 minutos e nuo aguentei jogar maisrealmente muito bom para jogar com o amigo entao e sensacional porem nao sou muito fa de jogos de carroo jogo o legal o problema o nuo ser bom neleputa jogo tiu  altos tiroteiosum dos melhores jogos sem duvidas nenhumasimplesmente um dos melhores jogos que joguei nos ultimos meses impecuvel na jogabilidade e no designquem noo comprou atu agora compre  fps tutico  gameplay diferente dos outros do genero\nin english below  port sempre tive um preconceito com esse jogo mas quando fui jogar por si su me surpreendi bastante vc se apega rupido com o niko ele u bem bacana as musicas boas e a imersuo suo garantidas ainda nuo zerei acabei de concertar o botuo do elevador se vc jogar vai saber e a ost da cidade meu amigo u incruvel obs o jogo nuo possuu batalhas mas vocu releva isso bem rupido por conta da histuria e a forma em que vocu avanca no jogo\n\neng i was aways

In [12]:
#Realizando o mesmo processo:
negativo = ''
opiniao_negativa = dados_treinamento.loc[dados_treinamento.Target == 'Negativo', ['review']]
for opiniao in opiniao_negativa.review:
    negativo += opiniao + ''

negativo


'jogo noo e ruim mas se eu pudesse eu voltava no passado e nao comprarianao posso avaliar porque não consigo jogar\njogo muito mal otimizado maior decepcao de 2116update dei outra chance comprei o rebirth e subitamente entendi a graca do jogo ^_^ mantenho o review negativo porque esta versuo em flash nem deveria mais ser vendida aqui no steam u rebirth na cabeca\n\nmeus amigos eu sei que vocus amam binding of isaac entuo perdoem a minha opiniuo altamente impopular mas eu acho esse negucio um saco\n\ndesculpem nuo u problema de git gud nuo u problema de vocu nuo entende roguelikes eu tive saco suficiente para me tornar bom em dark souls e ju investi boas horas em roguelikes de raiz como nethack o meu problema u com a jogabilidade acho a exploracuo chata acho comandar o personagem chato o jogo nuo me traz um pingo de satisfacuo por isso nuo posso recomendar o jogo mas se tanta gente diz que o jogo u fantustico recomendo que vocu experimente eu devo estar com algum parafuso soltoparabens 

Faremos agora:
1 - Guardar palavras com Pd.series
2 - Tabelar frequências absolutas
3 - Conjunto total de palavras

In [13]:
#Similarmente a aula 8, agora usa-se .split e .values para analisar as palavraas:
#Isso se faz, principalmente, para ver a repetição de palavras
tudo_ok = ok.split()
serie_ok = pd.Series(tudo_ok)
serie_ok.head(10)

0      targeted
1         ataca
2           com
3       numeros
4           nuo
5         ataca
6           com
7             o
8         mouse
9    mobilidade
dtype: object

In [14]:
#Agora com os negativos:
tudo_negativo = negativo.split()
serie_negativo = pd.Series(tudo_negativo)
serie_negativo.head(10)

0       jogo
1        noo
2          e
3       ruim
4        mas
5         se
6         eu
7    pudesse
8         eu
9    voltava
dtype: object

In [15]:
#Tendo as palavras listadas em série, podemos, assim, usar o value_counts() para contabilizar
#Tabela de frequência Classificação Ok
qtd_ok = serie_ok.value_counts()
qtd_ok

e           357
de          282
o           280
jogo        268
que         258
           ... 
busca         1
onlineem      1
partidas      1
formula       1
relatar       1
Length: 2883, dtype: int64

In [16]:
#Para os negativos:
#Tabela de frequência Classificação Negativo
qtd_negativo = serie_negativo.value_counts()
qtd_negativo

de             357
o              345
e              329
que            304
jogo           301
              ... 
event            1
participate      1
bucks            1
eventos          1
pissed           1
Length: 2979, dtype: int64

In [17]:
#Agora, com o objetivo de ter os dados separados da variável "classificação ok" e "classificação negativo", cria-se uma nova variável:
#Guardando total de palavras
todas_classificacoes = ok + negativo 
#Pensando agora na quantidade de palavras, vemos:
palavras_todas_classificacoes = todas_classificacoes.split()
serie_todas_classificacoes = pd.Series(palavras_todas_classificacoes)
serie_todas_classificacoes

0        targeted
1           ataca
2             com
3         numeros
4             nuo
           ...   
21676        isnt
21677       ready
21678          im
21679      pissed
21680        here
Length: 21681, dtype: object

In [18]:
#Similarmente ao que foi feito na aula 08 (Análise se é "Lusíadas" ou "Dom Casmurro", se da o início do cálculo de probabilidades)
p_ok = len(tudo_ok)/len(palavras_todas_classificacoes)
p_neg = len(tudo_negativo)/len(palavras_todas_classificacoes)

In [19]:
#Vendo as probabilidades das palavras:
print(p_ok, p_neg)

0.4825884414925511 0.5174576818412435


In [20]:
#Tendo isso, pode-se dar início - de fato - as contas probabilísticas.
#Usa-se um exemplo/Frase a classificar
#NaveBayes
ex = dados_treinamento.review[292]
#Separando
ex = ex.split()
ex

['me',
 'arrependo',
 'profundamente',
 'de',
 'ter',
 'platinado',
 'essa',
 'merda',
 'de',
 'jogo',
 'de',
 'depravado',
 'coomer',
 'do',
 'krl',
 'não',
 'compre',
 'a',
 'porra',
 'da',
 'dlc',
 'desse',
 'jogo',
 'kobra']

In [21]:
#Analisando as probabilidades:
import numpy #importando a biblioteca numpy
probabilidade_palavra_ok = []
for probabilidade in ex:
    if probabilidade in qtd_ok:
        operacao = (qtd_ok[probabilidade]+1)/(sum(qtd_ok)+len(serie_todas_classificacoes))
        probabilidade_palavra_ok.append(operacao)
    else:
        operacao = (0+1)/(sum(qtd_ok)+len(serie_todas_classificacoes))
        probabilidade_palavra_ok.append(operacao)

probabilidade_palavra_negativo = []
for probabilidade in ex:
    if probabilidade in qtd_negativo:
        operacao = (qtd_negativo[probabilidade]+1)/(sum(qtd_negativo)+len(serie_todas_classificacoes))
        probabilidade_palavra_negativo.append(operacao)
    else:
        operacao = (0+1)/(sum(qtd_negativo)+len(serie_todas_classificacoes))
        probabilidade_palavra_negativo.append(operacao)

probabilidade_palavra_ok = numpy.prod(probabilidade_palavra_ok)
probabilidade_palavra_negativo = numpy.prod(probabilidade_palavra_negativo)

print(probabilidade_palavra_ok, probabilidade_palavra_negativo)

7.6180891573559325e-84 1.725005728874521e-77


Agora, pode-se analisar as probabilidades:

In [22]:
if probabilidade_palavra_ok > probabilidade_palavra_negativo:
    print('Ok')
else:
    print('Negativo')

Negativo


De fato, o review da planilha de treinamento '292' é negativo. Logo, o código - de primeira nesse teste - funciona corretamente.

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [27]:
#Agora iremos refazer o processo que fizemos com a planilha de treinamento, mas, agora, na base de testes.
#Repetindo o código:

classificacoes_planilha_teste = test['review']
testados = [] #criação de lista com opiniões variadas entre ok e negativo (testadas)
for opiniao_teste in classificacoes_planilha_teste:
    opiniao_teste = opiniao_teste.split()
    probabilidade_palavra_ok = []
    for probabilidade in opiniao_teste:
        if probabilidade in qtd_ok:
            operacao = (qtd_ok[probabilidade]+1)/(sum(qtd_ok)+len(serie_todas_classificacoes))
            probabilidade_palavra_ok.append(operacao)
        else:
            operacao = (0+1)/(sum(qtd_ok)+len(serie_todas_classificacoes))
            probabilidade_palavra_ok.append(operacao)

    probabilidade_palavra_negativo = []
    for probabilidade in opiniao_teste:
        if probabilidade in qtd_negativo:
            operacao = (qtd_negativo[probabilidade]+1)/(sum(qtd_negativo)+len(serie_todas_classificacoes))
            probabilidade_palavra_negativo.append(operacao)
        else:
            operacao = (0+1)/(sum(qtd_negativo)+len(serie_todas_classificacoes))
            probabilidade_palavra_negativo.append(operacao)

    probabilidade_palavra_ok = numpy.prod(probabilidade_palavra_ok)
    probabilidade_palavra_negativo = numpy.prod(probabilidade_palavra_negativo)
    if probabilidade_palavra_negativo > probabilidade_palavra_ok:
        testados.append('Ok')
    else:
        testados.append('Negativo')

In [29]:
#Vendo, agora, a lista "Testados":
print(testados)

['Negativo', 'Ok', 'Ok', 'Ok', 'Ok', 'Negativo', 'Negativo', 'Ok', 'Ok', 'Negativo', 'Negativo', 'Negativo', 'Negativo', 'Ok', 'Ok', 'Negativo', 'Ok', 'Negativo', 'Ok', 'Negativo', 'Ok', 'Negativo', 'Ok', 'Ok', 'Negativo', 'Ok', 'Ok', 'Ok', 'Negativo', 'Ok', 'Ok', 'Ok', 'Negativo', 'Negativo', 'Ok', 'Negativo', 'Ok', 'Ok', 'Ok', 'Ok', 'Negativo', 'Ok', 'Negativo', 'Negativo', 'Negativo', 'Negativo', 'Negativo', 'Ok', 'Ok', 'Ok', 'Negativo', 'Ok', 'Negativo', 'Negativo', 'Ok', 'Negativo', 'Ok', 'Ok', 'Ok', 'Negativo', 'Ok', 'Negativo', 'Ok', 'Ok', 'Negativo', 'Ok', 'Ok', 'Ok', 'Ok', 'Negativo', 'Negativo', 'Ok', 'Ok', 'Ok', 'Negativo', 'Ok', 'Ok', 'Negativo', 'Negativo', 'Ok', 'Negativo', 'Ok', 'Ok', 'Ok', 'Negativo', 'Ok', 'Negativo', 'Ok', 'Ok', 'Ok', 'Ok', 'Negativo', 'Negativo', 'Ok', 'Negativo', 'Ok', 'Ok', 'Ok', 'Ok', 'Ok', 'Negativo', 'Ok', 'Negativo', 'Negativo', 'Ok', 'Ok', 'Ok', 'Ok', 'Ok', 'Negativo', 'Negativo', 'Negativo', 'Ok', 'Ok', 'Ok', 'Ok', 'Ok', 'Negativo', 'Ok', 'Ok

Agora, analisamos a nossa avaliação manual do target e, em seguida, fazemos nosso classificador:


In [30]:
target_avaliado = [] #criação de uma lista da avaliação
for i in test['Target']:
    target_avaliado.append(i)
print(target_avaliado)

[1, 1, 2, 1, 2, 1, 2, 2, 2, 1, 2, 2, 1, 2, 2, 1, 1, 1, 2, 2, 2, 1, 1, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 1, 1, 2, 2, 2, 1, 2, 1, 1, 1, 1, 2, 2, 2, 1, 1, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 1, 1, 1, 2, 1, 1, 2, 2, 2, 1, 2, 1, 2, 1, 2, 2, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 2, 2, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 2, 1, 2, 1, 1, 2, 1, 1, 1, 2, 2, 2, 1, 2, 2, 1, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 1, 2, 1, 2, 2, 2, 1, 2, 1, 2, 2, 2, 1, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 2, 1, 1, 2, 2, 1, 2, 2, 1, 2, 1, 2, 2, 1, 1, 2, 2, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 2, 2, 2, 2, 1, 1, 1, 2, 2, 1, 1, 2, 1, 2, 2, 2, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 2, 1, 2, 2, 1, 1, 1, 2, 2, 1, 2, 1, 1, 2, 2, 2, 1, 2, 2, 1, 2, 1, 1, 2, 2, 2, 1, 2, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 1, 2, 1, 2, 2, 1, 1, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 2, 1, 2, 1, 1, 1, 2, 2, 2, 2, 1, 1, 2]


In [ ]:
#Classificador


___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CONSIDEROU mais de duas categorias na variável Target e INCREMENTOU a quantidade de notícias, mantendo pelo menos 250 notícias por categoria (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* Para Target com duas categorias: CRIOU pelo menos quatro categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item Qualidade do Classificador a partir de novas separações das Notícias entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**